In [1]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
import time
import pandas as pd
from selenium.webdriver.common.by import By

In [2]:



def get_jobs(keyword, num_jobs, verbose, path, slp_time):
    
    '''Gathers jobs as a dataframe, scraped from Glassdoor'''
    
    #Initializing the webdriver
    options = webdriver.ChromeOptions()
    
    #Uncomment the line below if you'd like to scrape without a new Chrome window every time.
    #options.add_argument('headless')
    #Let the page load. Change this number based on your internet speed.
    #Or, wait until the webpage is loaded, instead of hardcoding it.
    options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(executable_path=path, options=options)
    driver.set_window_size(1120, 1000)
    
    url = "https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword="+keyword+"&sc.keyword="+keyword+"&locT=&locId=&jobType="
    #url = 'https://www.glassdoor.com/Job/jobs.htm?sc.keyword="' + keyword + '"&locT=C&locId=1147401&locKeyword=San%20Francisco,%20CA&jobType=all&fromAge=-1&minSalary=0&includeNoSalaryJobs=true&radius=100&cityId=-1&minRating=0.0&industryId=-1&sgocId=-1&seniorityType=all&companyId=-1&employerSizes=0&applicationType=0&remoteWorkType=0'
    driver.get(url)
    
    time.sleep(slp_time)

    jobs = []

    while len(jobs) < num_jobs:  #If true, should be still looking for new jobs.



         
        #Going through each job in this page
        #job_buttons = driver.find_elements(By.CSS_SELECTOR,"li[class^='JobsList_jobListItem']")  #jl for Job Listing. These are the buttons we're going to click.
        job_buttons = driver.find_elements(By.CSS_SELECTOR,"li[class^='JobsList_jobListItem']")
        print(len(job_buttons))
        for job_button in job_buttons: 

            print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))

            if len(jobs) >= num_jobs:
                break
           

            job_button.click()  #You might 
            time.sleep(1)
            collected_successfully = False


          
            while not collected_successfully:
                try:
                    company_name = job_button.find_element_by_xpath('.//span[@class="EmployerProfile_compactEmployerName__LE242"]').text
                    location = job_button.find_element_by_xpath('.//div[@class="JobCard_location__rCz3x"]').text
                    job_title = job_button.find_element_by_xpath('.//a[contains(@class, "JobCard_jobTitle___7I6y")]').text
                    job_description = job_button.find_element_by_xpath('.//div[@class="JobCard_jobDescriptionSnippet__yWW8q"]').text
                    collected_successfully = True
                except:
                    time.sleep(5)

            try:
                salary_estimate = job_button.find_element_by_xpath('.//div[@class="JobCard_salaryEstimate__arV5J"]').text
            except NoSuchElementException:
                salary_estimate = -1 #You need to set a "not found value. It's important."
            
            try:
                rating = job_button.find_element_by_xpath('.//div[@class="EmployerProfile_ratingContainer__ul0Ef"]').text
            except NoSuchElementException:
                rating = -1 #You need to set a "not found value. It's important."
            


            #Printing for debugging
            if verbose:
                print("Job Title: {}".format(job_title))
                print("Salary Estimate: {}".format(salary_estimate))
                print("Job Description: {}".format(job_description[:500]))
                print("Rating: {}".format(rating))
                print("Company Name: {}".format(company_name))
                print("Location: {}".format(location))
                
            



            #Going to the Company tab...
            #clicking on this:
            #<div class="tab" data-tab-type="overview"><span>Company</span></div>
            #try:
                #driver.find_element_by_xpath('.//div[@class="tab" and @data-tab-type="overview"]').click()
                

            try:
                size = driver.find_element(By.XPATH, "//div[@class='JobDetails_overviewItem__cAsry']//span[@class='JobDetails_overviewItemLabel__KjFln' and text()='Size']/following-sibling::div[@class='JobDetails_overviewItemValue__xn8EF']").text
            except NoSuchElementException:
                size = -1

            try:
                founded = driver.find_element(By.XPATH, "//div[@class='JobDetails_overviewItem__cAsry']//span[@class='JobDetails_overviewItemLabel__KjFln' and text()='Founded']/following-sibling::div[@class='JobDetails_overviewItemValue__xn8EF']").text
            except NoSuchElementException:
                founded = -1

            try:
                type_of_ownership = driver.find_element(By.XPATH, "//div[@class='JobDetails_overviewItem__cAsry']//span[@class='JobDetails_overviewItemLabel__KjFln' and text()='Type']/following-sibling::div[@class='JobDetails_overviewItemValue__xn8EF']").text
            except NoSuchElementException:
                type_of_ownership = -1

            try:
                industry = driver.find_element(By.XPATH, "//div[@class='JobDetails_overviewItem__cAsry']//span[@class='JobDetails_overviewItemLabel__KjFln' and text()='Industry']/following-sibling::div[@class='JobDetails_overviewItemValue__xn8EF']").text
            except NoSuchElementException:
                industry = -1

            try:
                sector = driver.find_element(By.XPATH, "//div[@class='JobDetails_overviewItem__cAsry']//span[@class='JobDetails_overviewItemLabel__KjFln' and text()='Sector']/following-sibling::div[@class='JobDetails_overviewItemValue__xn8EF']").text
            except NoSuchElementException:
                sector = -1

            try:
                revenue = driver.find_element(By.XPATH, "//div[@class='JobDetails_overviewItem__cAsry']//span[@class='JobDetails_overviewItemLabel__KjFln' and text()='Revenue']/following-sibling::div[@class='JobDetails_overviewItemValue__xn8EF']").text
            except NoSuchElementException:
                revenue = -1


            '''
            except NoSuchElementException:  #Rarely, some job postings do not have the "Company" tab.
                size = -1
                founded = -1
                type_of_ownership = -1
                industry = -1
                sector = -1
                revenue = -1

            '''
                
            if verbose:
                print("Size: {}".format(size))
                print("Founded: {}".format(founded))
                print("Type of Ownership: {}".format(type_of_ownership))
                print("Industry: {}".format(industry))
                print("Sector: {}".format(sector))
                print("Revenue: {}".format(revenue))
                print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")
         
            jobs.append({"Job Title" : job_title,
            "Salary Estimate" : salary_estimate,
            "Job Description" : job_description,
            "Rating" : rating,
            "Company Name" : company_name,
            "Location" : location,
            "Size" : size,
            "Founded" : founded,
            "Type of ownership" : type_of_ownership,
            "Industry" : industry,
            "Sector" : sector,
            "Revenue" : revenue})

            #add job to jobs
            
 
        #Clicking on the "next page" button
        try:
            driver.find_element_by_xpath('.//li[@class="next"]//a').click()
        except NoSuchElementException:
            print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
            break
        
    return pd.DataFrame(jobs)  #This line converts the dictionary object into a pandas DataFrame.

In [3]:
path = "E:\Projects\ds_salary_proj\chromedriver"
df = get_jobs('data scientist',90, False, path, 20)

121
Progress: 0/90
Progress: 1/90
Progress: 2/90
Progress: 3/90
Progress: 4/90
Progress: 5/90
Progress: 6/90
Progress: 7/90
Progress: 8/90
Progress: 9/90
Progress: 10/90
Progress: 11/90
Progress: 12/90
Progress: 13/90
Progress: 14/90
Progress: 15/90
Progress: 16/90
Progress: 17/90
Progress: 18/90
Progress: 19/90
Progress: 20/90
Progress: 21/90
Progress: 22/90
Progress: 23/90
Progress: 24/90
Progress: 25/90
Progress: 26/90
Progress: 27/90
Progress: 28/90
Progress: 29/90
Progress: 30/90
Progress: 31/90
Progress: 32/90
Progress: 33/90
Progress: 34/90
Progress: 35/90
Progress: 36/90
Progress: 37/90
Progress: 38/90
Progress: 39/90
Progress: 40/90
Progress: 41/90
Progress: 42/90
Progress: 43/90
Progress: 44/90
Progress: 45/90
Progress: 46/90
Progress: 47/90
Progress: 48/90
Progress: 49/90
Progress: 50/90
Progress: 51/90
Progress: 52/90
Progress: 53/90
Progress: 54/90
Progress: 55/90
Progress: 56/90
Progress: 57/90
Progress: 58/90
Progress: 59/90
Progress: 60/90
Progress: 61/90
Progress: 62/9

In [17]:
df

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Size,Founded,Type of ownership,Industry,Sector,Revenue
0,IT Tech II (Data Mining),USD 21.50 Per Hour (Employer est.),Valid driver’s license preferred. In this role...,-1,Exacore,"Iraan, TX",-1,-1,-1,-1,-1,-1
1,"Manager, Model Validation",USD 76K - 126K (Glassdoor est.),"5 years of experience in statistical modeling,...",4.1,Sallie Mae,"Newark, DE",1001 to 5000 Employees,1973,Company - Public,Banking & Lending,Financial Services,$1 to $5 billion (USD)
2,Senior Data Scientist,USD 200K (Employer est.),Demonstrated ability to effectively communicat...,4.5,Red Dog Media,"Henderson, NV",1 to 50 Employees,2008,Company - Private,Internet & Web Services,Information Technology,Unknown / Non-Applicable
3,Senior Biological Imaging Data Scientist,USD 150K - 175K (Employer est.),Experience with data science in technical/scie...,5.0,Thrive Bioscience,"Beverly, MA",1 to 50 Employees,--,Company - Private,Biotech & Pharmaceuticals,Pharmaceutical & Biotechnology,Unknown / Non-Applicable
4,Director of Advanced Analytics & Data Science,USD 104K - 142K (Glassdoor est.),Drive the development and deployment of disrup...,3.4,Hanesbrands,"Winston-Salem, NC",10000+ Employees,2005,Company - Public,Consumer Product Manufacturing,Manufacturing,$5 to $10 billion (USD)
...,...,...,...,...,...,...,...,...,...,...,...,...
85,Data Scientist,USD 73K - 166K (Employer est.),You’ll develop algorithms and systems and use ...,4.2,Booz Allen,"Huntsville, AL",10000+ Employees,1914,Company - Public,Business Consulting,Management & Consulting,$5 to $10 billion (USD)
86,Data Scientist 2465,USD 106K - 159K (Glassdoor est.),The Contractor shall use appropriate tools and...,4.4,Quevera LLC,"Fairfax, VA",51 to 200 Employees,2013,Company - Private,Enterprise Software & Network Solutions,Information Technology,Unknown / Non-Applicable
87,Data Scientist,USD 80K - 120K (Glassdoor est.),Progress towards or completion of a master’s d...,4.2,Southern Company,"Birmingham, AL",10000+ Employees,1945,Company - Public,Energy & Utilities,"Energy, Mining & Utilities",Unknown / Non-Applicable
88,Data Scientist,-1,Excellent written and verbal communication ski...,4.1,"Buyers Edge Platform, LLC",United States,Unknown,--,Company - Private,Information Technology Support Services,Information Technology,Unknown / Non-Applicable
